In [1]:
import numpy as np
from scipy.stats import poisson

Parameters:

In [2]:
alpha = 0.9
beta = 0.2
lambda_p = [1,5]
gamma = 0.1

Big gamma matrix:

In [3]:
big_gamma = np.array([[1- gamma, 0, gamma], [0, 1-gamma, gamma], [beta/2, beta/2, 1- beta]])
big_gamma

array([[0.9, 0. , 0.1],
       [0. , 0.9, 0.1],
       [0.1, 0.1, 0.8]])

Simulation function

In [4]:
def sim(n,T):
    C = [None]*T
    C[0] = 2
    C_trans = {(1,0,0): 0, (0,1,0): 1, (0,0,1): 2} 

    for i in range(1,T):
        draw = np.random.multinomial(1, big_gamma[C[i-1], :], 1)
        C[i] = C_trans[tuple(draw[0])]

    Z = np.empty([n, T])

    probs = {0: 1-alpha, 1: alpha, 2: 0.5}

    for i in range(n):
        for j in range(T):
            Z[i,j] = np.random.binomial(1, probs[C[j]])

    X = np.empty([n, T])

    for i in range(n):
        for j in range(T):
            X[i,j] = poisson.rvs(lambda_p[int(Z[i,j])])

    return C,Z,X

In [5]:
C, Z, X = sim(10,100)